In [1]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow as tf

from gcn.utils import *
from gcn.models import GCN, MLP

# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('dataset', 'cora', 'Dataset string.')  # 'cora', 'citeseer', 'pubmed'
flags.DEFINE_string('model', 'gcn', 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 16, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 10, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

In [2]:
# Load data
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(FLAGS.dataset)

In [3]:
adj

<2708x2708 sparse matrix of type '<class 'numpy.int64'>'
	with 10556 stored elements in Compressed Sparse Row format>

In [4]:
# Some preprocessing
features = preprocess_features(features)
if FLAGS.model == 'gcn':
    support = [preprocess_adj(adj)]
    print (support)
    num_supports = 1
    model_func = GCN
elif FLAGS.model == 'gcn_cheby':
    support = chebyshev_polynomials(adj, FLAGS.max_degree)
    num_supports = 1 + FLAGS.max_degree
    model_func = GCN
elif FLAGS.model == 'dense':
    support = [preprocess_adj(adj)]  # Not used
    num_supports = 1
    model_func = MLP
else:
    raise ValueError('Invalid argument for model: ' + str(FLAGS.model))

[(array([[   0,    0],
       [ 633,    0],
       [1862,    0],
       ...,
       [1473, 2707],
       [2706, 2707],
       [2707, 2707]], dtype=int32), array([0.25     , 0.25     , 0.2236068, ..., 0.2      , 0.2      ,
       0.2      ]), (2708, 2708))]


In [5]:
features

(array([[   0, 1274],
        [   0, 1247],
        [   0, 1194],
        ...,
        [2707,  329],
        [2707,  186],
        [2707,   19]], dtype=int32),
 array([0.11111111, 0.11111111, 0.11111111, ..., 0.07692308, 0.07692308,
        0.07692308], dtype=float32),
 (2708, 1433))

In [6]:
features[2][1]

1433

In [7]:
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

In [8]:
# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
# Initialize session
sess = tf.Session()

In [10]:
# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)

In [11]:
# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

In [12]:
# Train model
for epoch in range(FLAGS.epochs):
    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features, support, y_train, train_mask, placeholders)
    #print (feed_dict)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features, support, y_val, val_mask, placeholders)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

Epoch: 0001 train_loss= 1.95373 train_acc= 0.09286 val_loss= 1.95075 val_acc= 0.20200 time= 0.53712
Epoch: 0002 train_loss= 1.94812 train_acc= 0.32857 val_loss= 1.94717 val_acc= 0.34000 time= 0.11068
Epoch: 0003 train_loss= 1.94249 train_acc= 0.47857 val_loss= 1.94337 val_acc= 0.43000 time= 0.11135
Epoch: 0004 train_loss= 1.93550 train_acc= 0.55000 val_loss= 1.93957 val_acc= 0.46400 time= 0.10956
Epoch: 0005 train_loss= 1.92617 train_acc= 0.67143 val_loss= 1.93558 val_acc= 0.45400 time= 0.10993
Epoch: 0006 train_loss= 1.91827 train_acc= 0.65714 val_loss= 1.93151 val_acc= 0.45400 time= 0.11056
Epoch: 0007 train_loss= 1.90944 train_acc= 0.65714 val_loss= 1.92749 val_acc= 0.45600 time= 0.10662
Epoch: 0008 train_loss= 1.90143 train_acc= 0.65714 val_loss= 1.92358 val_acc= 0.45400 time= 0.10780
Epoch: 0009 train_loss= 1.89255 train_acc= 0.70000 val_loss= 1.91966 val_acc= 0.46600 time= 0.10790
Epoch: 0010 train_loss= 1.88016 train_acc= 0.70000 val_loss= 1.91560 val_acc= 0.48200 time= 0.11098


Epoch: 0083 train_loss= 0.97041 train_acc= 0.90000 val_loss= 1.37076 val_acc= 0.78000 time= 0.10939
Epoch: 0084 train_loss= 1.01846 train_acc= 0.93571 val_loss= 1.36531 val_acc= 0.78000 time= 0.11585
Epoch: 0085 train_loss= 0.99078 train_acc= 0.90000 val_loss= 1.35981 val_acc= 0.78000 time= 0.11754
Epoch: 0086 train_loss= 0.95950 train_acc= 0.95000 val_loss= 1.35422 val_acc= 0.78000 time= 0.12747
Epoch: 0087 train_loss= 0.98173 train_acc= 0.94286 val_loss= 1.34867 val_acc= 0.78000 time= 0.13487
Epoch: 0088 train_loss= 0.94133 train_acc= 0.95000 val_loss= 1.34307 val_acc= 0.78000 time= 0.13006
Epoch: 0089 train_loss= 0.94469 train_acc= 0.91429 val_loss= 1.33738 val_acc= 0.77800 time= 0.14461
Epoch: 0090 train_loss= 0.93210 train_acc= 0.90714 val_loss= 1.33178 val_acc= 0.77800 time= 0.12580
Epoch: 0091 train_loss= 0.94818 train_acc= 0.94286 val_loss= 1.32622 val_acc= 0.77800 time= 0.13268
Epoch: 0092 train_loss= 0.96379 train_acc= 0.93571 val_loss= 1.32098 val_acc= 0.77800 time= 0.12030


Epoch: 0165 train_loss= 0.71285 train_acc= 0.95000 val_loss= 1.09738 val_acc= 0.78000 time= 0.11122
Epoch: 0166 train_loss= 0.66846 train_acc= 0.95000 val_loss= 1.09619 val_acc= 0.78000 time= 0.11391
Epoch: 0167 train_loss= 0.65185 train_acc= 0.97143 val_loss= 1.09502 val_acc= 0.78000 time= 0.11717
Epoch: 0168 train_loss= 0.65922 train_acc= 0.95000 val_loss= 1.09437 val_acc= 0.78000 time= 0.10786
Epoch: 0169 train_loss= 0.63922 train_acc= 0.98571 val_loss= 1.09393 val_acc= 0.78000 time= 0.11038
Epoch: 0170 train_loss= 0.65303 train_acc= 0.95000 val_loss= 1.09337 val_acc= 0.78400 time= 0.10880
Epoch: 0171 train_loss= 0.71675 train_acc= 0.95000 val_loss= 1.09231 val_acc= 0.78400 time= 0.11562
Epoch: 0172 train_loss= 0.60315 train_acc= 0.96429 val_loss= 1.09168 val_acc= 0.78400 time= 0.11160
Epoch: 0173 train_loss= 0.66708 train_acc= 0.95000 val_loss= 1.09123 val_acc= 0.78200 time= 0.11154
Epoch: 0174 train_loss= 0.60544 train_acc= 0.96429 val_loss= 1.09062 val_acc= 0.78400 time= 0.11149


In [13]:
feed_dict

{<tf.Tensor 'Placeholder_5:0' shape=(?, 7) dtype=float32>: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 <tf.Tensor 'Placeholder_6:0' shape=<unknown> dtype=int32>: array([ True,  True,  True, ..., False, False, False]),
 <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x10a2d1a90>: (array([[   0, 1274],
         [   0, 1247],
         [   0, 1194],
         ...,
         [2707,  329],
         [2707,  186],
         [2707,   19]], dtype=int32),
  array([0.11111111, 0.11111111, 0.11111111, ..., 0.07692308, 0.07692308,
         0.07692308], dtype=float32),
  (2708, 1433)),
 <tensorflow.python.framework.sparse_tensor.SparseTensor at 0xd36aa0e10>: (array([[   0,    0],
         [ 633,    0],
         [1862,    0],
         ...,
         [1473, 2707],
         [2706, 2707],
      

In [14]:
# Testing
test_cost, test_acc, test_duration = evaluate(features, support, y_test, test_mask, placeholders)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))

Test set results: cost= 1.00983 accuracy= 0.81300 time= 0.05553
